In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy
import scipy.stats as stats
import seaborn.apionly as sns
import statsmodels.api as sm
import theano.tensor as tt
import pymc3
from sklearn import preprocessing
%matplotlib inline

/Users/dgoldma1/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Load in 2016 weather data.

- station identifier (GHCN Daily Identification Number)

 - date (yyyymmdd; where yyyy=year; mm=month; and, dd=day)
 - observation type (see ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt for definitions)
 - observation value (see ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt for units)
 - observation time (if available, as hhmm where hh=hour and mm=minutes in local time)


In [3]:
df_weather = pd.read_csv('../data/weather/2016_weather.csv',names=['airport_code','date','type','value','t1','t2','t3','t4'])

In [68]:
df_weather.head()

,airport_code,date,type,value,t1,t2,t3,t4
0,US1NJES0019,20160101,PRCP,0,NaN,NaN,N,NaN
1,US1NJES0019,20160101,SNOW,0,NaN,NaN,N,NaN
2,US1NJGL0001,20160101,PRCP,0,NaN,NaN,N,NaN
3,US1NJGL0001,20160101,SNOW,0,NaN,NaN,N,NaN
4,CA1AB000023,20160101,PRCP,0,NaN,NaN,N,NaN


According to the readme, the value -999 is used only when there is no record available.

In [4]:
df_weather=df_weather[df_weather['value'] > -999]

## Load in airport lookup data.

The format was bad as it was seperated by spaces and not tabs. So I did the following bash command to make a tsv:

In [59]:
#!cat stations_airport.txt | awk {'printf ("%s\t%s\t%s\t%s\t%s %s %s %s %s\n", $1, $2,$3,$4,$5,$6,$7,$8,$9)'} > airports.tsv

In [5]:
df_stations=pd.read_csv('../data/weather/airports.tsv',sep='\t',names=['airport_code','lat','lon','height','name'])

In [74]:
df_stations.head()

,airport_code,lat,lon,height,name
0,AM000037699,40.5330,44.3830,1892.0,APARAN 37699
1,AQC00914869,-14.3333,-170.7167,3.0,AS TAFUNA AP TUTUILA
2,AQW00061705,-14.3306,-170.7136,3.7,AS PAGO PAGO WSO AP
3,ASN00017006,-29.9267,138.7517,123.0,APOLLINARIS WELL
4,ASN00019001,-33.0524,138.4277,369.0,APPILA


In [75]:
df_weather.head()

,airport_code,date,type,value,t1,t2,t3,t4
0,US1NJES0019,20160101,PRCP,0,NaN,NaN,N,NaN
1,US1NJES0019,20160101,SNOW,0,NaN,NaN,N,NaN
2,US1NJGL0001,20160101,PRCP,0,NaN,NaN,N,NaN
3,US1NJGL0001,20160101,SNOW,0,NaN,NaN,N,NaN
4,CA1AB000023,20160101,PRCP,0,NaN,NaN,N,NaN


### Join the weather data to the location data

In [6]:
df_weather_stations = pd.merge(df_stations,df_weather,how='inner',on=['airport_code'])

In [81]:
df_weather_stations

,airport_code,lat,lon,height,name,date,type,value,t1,t2,t3,t4
0,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160101,TMAX,-80,NaN,NaN,S,NaN
1,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160101,TMIN,-113,NaN,NaN,S,NaN
2,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160101,PRCP,61,NaN,NaN,S,NaN
3,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160101,TAVG,-104,H,NaN,S,NaN
4,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160102,TMAX,-96,NaN,NaN,S,NaN
5,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160102,PRCP,61,NaN,NaN,S,NaN
6,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160102,TAVG,-137,H,NaN,S,NaN
7,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160103,TMAX,-136,NaN,NaN,S,NaN
8,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160103,TMIN,-230,NaN,NaN,S,NaN
9,AM000037699,40.5330,44.3830,1892.0,APARAN 37699,20160103,PRCP,0,NaN,NaN,S,NaN


In [7]:
df_codelatlon = pd.read_csv('../data/airport_data/airport_codes_latlon.csv')

### Fix lat lon and join to get locationID of weather statoin which can be joined to the on time performance dataset.

In [88]:
df_weather_stations[df_weather_stations['name'].apply(lambda x : x.find('JFK')>-1)].head()

,airport_code,lat,lon,height,name,date,type,value,t1,t2,t3,t4
2684993,USW00094789,40.6386,-73.7622,3.4,NY NEW YORK JFK INTL,20160101,TMAX,67,NaN,NaN,W,2400.0
2684994,USW00094789,40.6386,-73.7622,3.4,NY NEW YORK JFK INTL,20160101,TMIN,22,NaN,NaN,W,2400.0
2684995,USW00094789,40.6386,-73.7622,3.4,NY NEW YORK JFK INTL,20160101,PRCP,0,NaN,NaN,W,2400.0
2684996,USW00094789,40.6386,-73.7622,3.4,NY NEW YORK JFK INTL,20160101,SNOW,0,NaN,NaN,W,NaN
2684997,USW00094789,40.6386,-73.7622,3.4,NY NEW YORK JFK INTL,20160101,SNWD,0,NaN,NaN,W,NaN


In [86]:
df_weather_stations[(df_weather_stations['lat']==40.6397) & (df_weather_stations['lon']==73.7789)]

,airport_code,lat,lon,height,name,date,type,value,t1,t2,t3,t4


### Load in On Time Performance Data

In [50]:
df_air=pd.read_csv('../data/On_Time_Performance/On_Time_Performance_2016_agg.tsv',sep='\t')

In [27]:
df_weather_stations['lon'].max()

171.40000000000001

## Join data on latitude and longitude 

In [28]:
df_weather_stations['lat']=df_weather_stations['lat'].apply(lambda x : abs(round(x,1)))
df_weather_stations['lon']=df_weather_stations['lon'].apply(lambda x : abs(round(x,1)))

In [29]:
df_codelatlon['lat']=df_codelatlon['Latitude'].apply(lambda x : abs(round(x,1)))
df_codelatlon['lon']=df_codelatlon['Longitude'].apply(lambda x : abs(round(x,1)))

In [30]:
df_joined=pd.merge(df_weather_stations,df_codelatlon,how='inner',on=['lat','lon'])

In [33]:
df_joined.fillna(0,inplace=True)

In [37]:
df_joined[df_joined['locationID']=='LGA']

,airport_code,lat,lon,height,name,date,type,value,t1,t2,t3,t4,locationID,Latitude,Longitude
1004722,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,TMAX,61,0,0,W,2400.0,LGA,40.7772,73.8725
1004723,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,TMIN,22,0,0,W,2400.0,LGA,40.7772,73.8725
1004724,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,PRCP,0,0,0,W,2400.0,LGA,40.7772,73.8725
1004725,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,SNOW,0,0,0,W,0.0,LGA,40.7772,73.8725
1004726,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,SNWD,0,0,0,W,0.0,LGA,40.7772,73.8725
1004727,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,AWND,60,0,0,W,0.0,LGA,40.7772,73.8725
1004728,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,TAVG,55,H,0,S,0.0,LGA,40.7772,73.8725
1004729,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,WDF2,260,0,0,W,0.0,LGA,40.7772,73.8725
1004730,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,WDF5,260,0,0,W,0.0,LGA,40.7772,73.8725
1004731,USW00014732,40.8,73.9,3.4,NY NEW YORK LAGUARDIA AP,20160101,WSF2,103,0,0,W,0.0,LGA,40.7772,73.8725


In [38]:
df_joined.to_csv('../data/weather/weather2016_row_airport_code.tsv',sep='\t',index=False)

### Convert to record based format

In [40]:
df_record =pd.concat([df_joined,pd.get_dummies(df_joined['type'])],axis=1)

In [43]:
df_record.head(2)

,airport_code,lat,lon,height,name,date,type,value,t1,t2,...,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11
0,AQW00061705,14.3,170.7,3.7,AS PAGO PAGO WSO AP,20160101,TMAX,286,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AQW00061705,14.3,170.7,3.7,AS PAGO PAGO WSO AP,20160101,TMIN,260,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
def fill_value(row):
    tmp_val = row['value']
    row[row['type']]=tmp_val
    return row


df_record=df_record.apply(lambda row : fill_value(row),axis=1)

In [47]:
df_record.to_csv('../data/weather/weather2016_row_airport_code_wdummies.tsv',sep='\t',index=False)

# Aggregation

In [49]:
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [2]:
#df_record.head(2)

In [54]:
weather_cols=df_record['type'].drop_duplicates().values

In [57]:
df_agg=df_record.groupby(['locationID','airport_code','lat','lon','date'])[weather_cols].sum()

In [59]:
df_agg.to_csv('../data/weather/all_weather_data_agg.tsv',sep='\t')

### Reload 

In [60]:
df_agg=pd.read_csv('../data/weather/all_weather_data_agg.tsv',sep='\t')

In [62]:
df_agg[df_agg['locationID']=='JFK']

,locationID,airport_code,lat,lon,date,TMAX,TMIN,PRCP,AWND,TAVG,WDF2,WDF5,WSF2,WSF5,TOBS,SNOW,SNWD,WT01,WT08,WT03,WT02,PSUN,TSUN,PGTM,WESD,WESF,WT05,WT06,WT04,WT11,WT09,DAPR,MDPR,SN32,SX32,WT07,WT10,SN52,SX52,EVAP,MNPN,MXPN,WDMV
165206,JFK,USW00094789,40.6,73.8,20160101,67,22,0,65,58,290,290,125,148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165207,JFK,USW00094789,40.6,73.8,20160102,50,11,0,58,31,270,260,107,134,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165208,JFK,USW00094789,40.6,73.8,20160103,89,17,0,63,46,290,290,103,130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165209,JFK,USW00094789,40.6,73.8,20160104,33,-93,0,89,10,350,350,139,174,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165210,JFK,USW00094789,40.6,73.8,20160105,-16,-110,0,55,-72,360,350,125,170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165211,JFK,USW00094789,40.6,73.8,20160106,44,-60,0,34,-14,230,230,63,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165212,JFK,USW00094789,40.6,73.8,20160107,78,-27,0,17,27,270,270,58,67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165213,JFK,USW00094789,40.6,73.8,20160108,83,0,0,40,43,50,40,72,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165214,JFK,USW00094789,40.6,73.8,20160109,117,44,5,52,72,90,90,94,116,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
165215,JFK,USW00094789,40.6,73.8,20160110,144,50,325,94,112,90,90,157,197,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_planes = pd.read_csv('../data/airport_data/plane-data.csv')

In [ ]:
#df_planes.head()